# Data Wrangling
In this notebook we transform the original data to the appropriate structure, by making transformations, creating new variables, dropping variables and correcting missing values.

The database has 15781 registers

In [1]:
#Libraries
import pandas as pd                  # Data Manipulation
import numpy as np                   # Matrices support
import matplotlib.pyplot as plt      # Plotting
import seaborn as sns                # Statistical Visualization
from matplotlib import cm
from scipy import stats
from numpy.polynomial.polynomial import polyfit
from random import randint
import math

In [2]:
data = pd.read_excel('201713_ECOAS_Anonimizado.xlsx')

## Drops variables
Multiple columns have only one value or are of no interest

In [3]:
drop_variables = ['Ejercicio','Grupo Virtual','Crn UV']
data = data.drop(drop_variables,axis=1)

## Replace nan with 0s
Now that we have the useful variables, we are going to give them the appropriate format.

In [4]:
#Replace nan values of multiple columns to 0s
nan_list = ['Tesis en título','Book','Book Series','Conference Proceeding','Journal Q1/Q2','Journal Q3/Q4','Total Docs','Book Norm.',
            'Book Series Norm.','Conference Proceeding Norm.','Journal Q1/Q2 Norm.','Journal Q3/Q4 Norm.',
            'Total Docs. Norm.']

for index,value in enumerate(nan_list):
    data[value] = data[value].fillna(0)

## Rename variables

In [5]:
#Renames variables
data.rename(columns = {'ID_Prof':'ID'}, inplace = True)
data.rename(columns = {'Rectoría del Campus':'rectoria'}, inplace = True)
data.rename(columns = {'Campus':'campus'}, inplace = True)
data.rename(columns = {'División':'division'}, inplace = True)
data.rename(columns = {'Departamento':'department'}, inplace = True)
data.rename(columns = {'Tesis en título':'thesis'}, inplace = True)
data.rename(columns = {'Número de Grupo':'num_grup'}, inplace = True)
data.rename(columns = {'Categoría Laboral':'cat_lab'}, inplace = True)
data.rename(columns = {'Puesto (Escolar)':'cat_escolar'}, inplace = True)
data.rename(columns = {'Puesto (RH)':'cat_rh'}, inplace = True)
data.rename(columns = {'Nacionalidad':'nationality'}, inplace = True)
data.rename(columns = {'Género':'gender'}, inplace = True)
data.rename(columns = {'Edad al semestre':'age'}, inplace = True)
data.rename(columns = {'Años de Antigüedad':'antiguedad'}, inplace = True)
data.rename(columns = {'Grado Máximo Prof':'degree'}, inplace = True)
data.rename(columns = {'Estatus PDHD (Habilidades Docentes)':'pdhd'}, inplace = True)
data.rename(columns = {'Semestres Experiencia':'experience'}, inplace = True)

data.rename(columns = {'Escuela Nacional Disciplina':'school_national'}, inplace = True)
data.rename(columns = {'Escuela GIEE':'school_giee'}, inplace = True)
data.rename(columns = {'Giee':'giee'}, inplace = True)
data.rename(columns = {'Participación en GIEE':'giee_role'}, inplace = True)
data.rename(columns = {'Nivel SNI 2019':'SNI'}, inplace = True)
data.rename(columns = {'Porcentaje responsabilidad prof. grupo':'percentage_responsability'}, inplace = True)
data.rename(columns = {'Nivel materia':'level'}, inplace = True)
data.rename(columns = {'Indicador grupo terminal':'terminal_group'}, inplace = True)
data.rename(columns = {'Atributos grupo':'course_attributes'}, inplace = True)
data.rename(columns = {'# Multicampus':'multicampus'}, inplace = True)
data.rename(columns = {'Profesor Titular':'main_professor'}, inplace = True)
data.rename(columns = {'Alumnos Candidatos':'seniors'}, inplace = True)
data.rename(columns = {'Alumnos que Opinaron':'opinions'}, inplace = True)
data.rename(columns = {'Participación (%)':'participation'}, inplace = True)
data.rename(columns = {'Horas Equivalentes Frente al Grupo':'teach_hours'}, inplace = True)
data.rename(columns = {'Horas clase':'lecture_hours'}, inplace = True)
data.rename(columns = {'Horas laboratorio':'lab_hours'}, inplace = True)
data.rename(columns = {'Unidades Equivalentes':'credits'}, inplace = True)

#40-48
data.rename(columns = {'05 #Op':'05num','05 Prom':'05avg','05 Desv':'05std','06 #Op':'06num','06 Prom':'06avg','06 Desv':'06std','08 #Op':'08num','08 Prom':'08avg','08 Desv':'08std'},inplace=True)

data.rename(columns = {'Book':'books'}, inplace = True)
data.rename(columns = {'Book Series':'bseries'}, inplace = True)
data.rename(columns = {'Conference Proceeding':'proceedings'}, inplace = True)
data.rename(columns = {'Journal Q1/Q2':'journal_q1q2'}, inplace = True)
data.rename(columns = {'Journal Q3/Q4':'journal_q3q4'}, inplace = True)
data.rename(columns = {'Total Docs':'total'}, inplace = True)
data.rename(columns = {'Book Norm.':'books_norm'}, inplace = True)
data.rename(columns = {'Book Series Norm.':'bseries_norm'}, inplace = True)
data.rename(columns = {'Conference Proceeding Norm.':'proceedings_norm'}, inplace = True)
data.rename(columns = {'Journal Q1/Q2 Norm.':'journal_q1q2_norm'}, inplace = True)
data.rename(columns = {'Journal Q3/Q4 Norm.':'journal_q1q2_norm'}, inplace = True)
data.rename(columns = {'Total Docs. Norm.':'total_norm'}, inplace = True)


# Drop data
Drop subjects marked as thesis

In [6]:
data = data[data['thesis']!=1].reset_index(drop=True)
data = data.drop(['thesis'],axis=1)

## Create variables

## sni_yn

In [7]:
#Creates a count table of sni levels 
sni = data[['SNI','ID']].groupby(['SNI'],as_index=False).count()
sni['percentage'] = sni['ID']/len(data)*100

#10% of profesors are reseearchers (SNI). So we create a variable named sni_yn (sni yes or no)
a = [];
num = data['SNI'].isnull()*1
for index,value in enumerate(num):
    if value == 0:
        a.append('SNI')
    else:
        a.append('NO SNI')
data['sni_yn'] = a                                                                  #Creates the new variable

## calif

In [8]:
#Creates the variable "calif" which is the average of the answers of the questions in the survery
data['score'] = (data['05avg'] + data['06avg'] + data['08avg'])/3

## under_yn

In [9]:
#Creates the variable "under_yn" which classifies if a class is at undergraduate level or not
#We grouped Doctorado, Maestría and Especialidad as Posgrado
a = [];
for index,value in enumerate(data['level']):
    if value == 'Profesional':
        a.append('Undergraduate')
    elif value == 'Preparatoria':
        a.append('Highschool')
    else:
        a.append('Graduate')
data['under_yn'] = a

## %Subjects by level

In [11]:
# Creates a series of ID and the number of times it appears, that is, the number of classes each professor has
counts_id = data.ID.value_counts()

In [12]:
# Creates a data frame of the previous series
counts_id_df = pd.DataFrame({'ID':counts_id.index,'counts_total':counts_id.values})

In [14]:
data = data.merge(counts_id_df, how='left', left_on='ID', right_on='ID')

## Graduate courses by professor

In [16]:
counts_id_graduate = data[data['under_yn']=='Graduate'].ID.value_counts()

In [17]:
counts_id_graduate_df = pd.DataFrame({'ID':counts_id_graduate.index,'counts_graduate':counts_id_graduate.values})

In [18]:
data = data.merge(counts_id_graduate_df, how='left', left_on='ID', right_on='ID')

## Undergraduate courses by professor

In [20]:
counts_id_undergraduate = data[data['under_yn']=='Undergraduate'].ID.value_counts()

In [21]:
counts_id_undergraduate_df = pd.DataFrame({'ID':counts_id_undergraduate.index,'counts_undergraduate':counts_id_undergraduate.values})

In [22]:
data = data.merge(counts_id_undergraduate_df, how='left', left_on='ID', right_on='ID')

## Highschool courses by professor

In [24]:
counts_id_highschool = data[data['under_yn']=='Highschool'].ID.value_counts()

In [25]:
counts_id_highschool_df = pd.DataFrame({'ID':counts_id_highschool.index,'counts_highschool':counts_id_highschool.values})

In [26]:
data = data.merge(counts_id_highschool_df, how='left', left_on='ID', right_on='ID')

## Remove nan

In [29]:
#Replace nan values of multiple columns to 0s
nan_list = ['counts_total','counts_graduate','counts_undergraduate','counts_highschool']

for index,value in enumerate(nan_list):
    data[value] = data[value].fillna(0)

## mxn_yn

In [37]:
#Counts the amount of profesors per nationality
prof_nacionalidad =  data[['nationality','ID']].groupby(['nationality'],as_index=False).count().sort_values(['ID'],ascending=False)
prof_nacionalidad['percentage'] = prof_nacionalidad['ID']/len(data)*100

#Because 94% of the professors are mexican, we create a variable named mxn_yn
a = [];
for index,value in enumerate(data['nationality']):
    if value == 'Mexicana':
        a.append('Mexican');
    else:
        a.append('Foreign');
data['mxn_yn'] = a        

In [43]:
#Changes the data type of colums to categorical
data['sni_yn'] = data['sni_yn'].astype('category')
data['under_yn'] = data['under_yn'].astype('category')
data['mxn_yn'] = data['mxn_yn'].astype('category')
data['pdhd'] = data['pdhd'].astype('category')
data['gender'] = data['gender'].astype('category')
data['terminal_group'] = data['terminal_group'].astype('category')
data['main_professor'] = data['main_professor'].astype('category')